In [4]:
def point_in_polygon(point, polygon):
    """
    Determine if a point is inside a polygon using the Ray Casting method.
    
    Parameters:
    - point (list): [longitude, latitude]
    - polygon (list of list): [[longitude, latitude], ...]

    Returns:
    - bool: True if point is inside the polygon, otherwise False
    """
    
    x, y = point
    odd_nodes = False
    j = len(polygon) - 1  # The last vertex is the previous one to the first

    for i in range(len(polygon)):
        xi, yi = polygon[i]
        xj, yj = polygon[j]
        if yi < y and yj >= y or yj < y and yi >= y: 
            if xi + (y - yi) / (yj - yi) * (xj - xi) < x:
                odd_nodes = not odd_nodes
        j = i

    return odd_nodes


False

In [72]:
import os
import json
import copy

def align_geojson_to_reference(data, reference):
    """Align the QGIS GeoJSON data to match the structure of the reference."""
    if isinstance(reference, dict):
        # If the key exists in the reference but not in the data, add it with NULL value
        for key, value in reference.items():
            if key not in data:
                data[key] = None if not isinstance(value, (dict, list)) else copy.deepcopy(value)
            else:
                data[key] = align_geojson_to_reference(data[key], value)

        # If the key exists in the data but not in the reference, remove it
        extra_keys = set(data.keys()) - set(reference.keys())
        for key in extra_keys:
            del data[key]
    elif isinstance(reference, list) and len(reference) > 0:
        # Handle the features list as a special case
        # Apply the reference structure to each feature in the data
        ref_item = reference[0]
        for idx, item in enumerate(data):
            data[idx] = align_geojson_to_reference(item, ref_item)

    return data

def preprocess_geojsons(qgis_file_path, reference_file_path):
    """Preprocess the QGIS GeoJSON based on the reference sample."""
    with open(qgis_file_path, 'r') as qgis_file:
        qgis_data = json.load(qgis_file)
    with open(reference_file_path, 'r') as ref_file:
        reference_data = json.load(ref_file)
    
    aligned_data = align_geojson_to_reference(qgis_data, reference_data)
    return aligned_data

def set_feature_type(data, feature_type_name):
    """Set the feature_type for each feature in the GeoJSON data."""
    for feature in data.get("features", []):
        feature["feature_type"] = feature_type_name
        feature["type"] = "Feature"


    return data


def process_all_geojsons(input_dir, output_dir, reference_dir):
    """Process all GeoJSONs in the input directory to align them to the reference format."""
    for file_name in os.listdir(reference_dir):
        if file_name.endswith(".geojson"):
            qgis_file_path = os.path.join(input_dir, file_name)
            reference_file_path = os.path.join(reference_dir, file_name)
            
            if os.path.exists(qgis_file_path):  # If QGIS GeoJSON exists
                aligned_data = preprocess_geojsons(qgis_file_path, reference_file_path)
            else:  # If QGIS GeoJSON does not exist, just copy the reference
                with open(reference_file_path, 'r') as ref_file:
                    aligned_data = json.load(ref_file)
            
            # Set the feature_type for each feature to be the same as the name key
            feature_type_name = aligned_data.get("name", "")
            set_feature_type(aligned_data, feature_type_name)

            # Save the aligned data
            output_file_path = os.path.join(output_dir, file_name)
            with open(output_file_path, 'w') as output_file:
                json.dump(aligned_data, output_file, indent=4)
            print(f"Processed {qgis_file_path} -> {output_file_path}")


if __name__ == "__main__":
    input_dir = "C:\\Users\\amosk\\GitHub\\Geojson_2_IMDF_Transformer\\QGIS_Geojsons"
    output_dir = "C:\\Users\\amosk\\GitHub\\Geojson_2_IMDF_Transformer\\Transformed_QGIS_Geojsons"
    reference_dir = "C:\\Users\\amosk\\GitHub\\Geojson_2_IMDF_Transformer\\IMDF_Geojson_Cleaned_Sample"
    process_all_geojsons(input_dir, output_dir, reference_dir)


Processed C:\Users\amosk\GitHub\Geojson_2_IMDF_Transformer\QGIS_Geojsons\address.geojson -> C:\Users\amosk\GitHub\Geojson_2_IMDF_Transformer\Transformed_QGIS_Geojsons\address.geojson
Processed C:\Users\amosk\GitHub\Geojson_2_IMDF_Transformer\QGIS_Geojsons\amenity.geojson -> C:\Users\amosk\GitHub\Geojson_2_IMDF_Transformer\Transformed_QGIS_Geojsons\amenity.geojson
Processed C:\Users\amosk\GitHub\Geojson_2_IMDF_Transformer\QGIS_Geojsons\anchor.geojson -> C:\Users\amosk\GitHub\Geojson_2_IMDF_Transformer\Transformed_QGIS_Geojsons\anchor.geojson
Processed C:\Users\amosk\GitHub\Geojson_2_IMDF_Transformer\QGIS_Geojsons\building.geojson -> C:\Users\amosk\GitHub\Geojson_2_IMDF_Transformer\Transformed_QGIS_Geojsons\building.geojson
Processed C:\Users\amosk\GitHub\Geojson_2_IMDF_Transformer\QGIS_Geojsons\footprint.geojson -> C:\Users\amosk\GitHub\Geojson_2_IMDF_Transformer\Transformed_QGIS_Geojsons\footprint.geojson
Processed C:\Users\amosk\GitHub\Geojson_2_IMDF_Transformer\QGIS_Geojsons\level.ge

In [73]:
import json
import uuid
import os

def add_uuid_to_geojson(file_path):
    """
    Add a UUID to the "id" key of every feature in the given GeoJSON file.
    """
    # Load the GeoJSON data from the file
    with open(file_path, 'r') as f:
        data = json.load(f)
    
    # Iterate through each feature and add a UUID to the "id" key
    for feature in data.get('features', []):
        feature['id'] = str(uuid.uuid4())
    
    # Save the modified GeoJSON back to the file
    with open(file_path, 'w') as f:
        json.dump(data, f, indent=4)

def main():
    # Directory containing the GeoJSON files
    directory_path = "C:\\Users\\amosk\\GitHub\\Geojson_2_IMDF_Transformer\\Transformed_QGIS_Geojsons"

    # Iterate over all files in the directory and process those with .geojson extension
    for filename in os.listdir(directory_path):
        if filename.endswith(".geojson"):
            file_path = os.path.join(directory_path, filename)
            add_uuid_to_geojson(file_path)
            print(f"UUIDs added successfully to {filename}")

if __name__ == "__main__":
    main()


UUIDs added successfully to address.geojson
UUIDs added successfully to amenity.geojson
UUIDs added successfully to anchor.geojson
UUIDs added successfully to building.geojson
UUIDs added successfully to footprint.geojson
UUIDs added successfully to level.geojson
UUIDs added successfully to occupant.geojson
UUIDs added successfully to opening.geojson
UUIDs added successfully to unit.geojson
UUIDs added successfully to venue.geojson


In [77]:
import json
import os

def get_uuid_from_geojson(file_path, feature_type):
    """
    Extract UUID from the given GeoJSON file based on the feature type.
    """
    with open(file_path, 'r') as f:
        data = json.load(f)

    for feature in data.get('features', []):
        if feature.get('feature_type') == feature_type:
            return feature.get('id')
    return None

def update_ids_in_geojson(file_path, address_uuid, building_uuid, level_uuid):
    """
    Update the address_id, building_ids, and level_id in the given GeoJSON file with the provided UUIDs.
    """
    with open(file_path, 'r') as f:
        data = json.load(f)

    for feature in data.get('features', []):
        # Only replace the keys if they already exist in the properties
        if 'address_id' in feature['properties']:
            feature['properties']['address_id'] = address_uuid
        if 'building_ids' in feature['properties']:
            feature['properties']['building_ids'] = [building_uuid]
        if 'level_id' in feature['properties']:
            feature['properties']['level_id'] = level_uuid

    with open(file_path, 'w') as f:
        json.dump(data, f, indent=4)


def main():
    directory_path = "C:\\Users\\amosk\\GitHub\\Geojson_2_IMDF_Transformer\\Transformed_QGIS_Geojsons"

    # Extract UUIDs from address, building, and level geojsons
    address_uuid = get_uuid_from_geojson(os.path.join(directory_path, 'address.geojson'), 'address')
    building_uuid = get_uuid_from_geojson(os.path.join(directory_path, 'building.geojson'), 'building')
    level_uuid = get_uuid_from_geojson(os.path.join(directory_path, 'level.geojson'), 'level')

    # List of GeoJSON files that need to be updated
    geojson_files_to_update = ['level.geojson', 'venue.geojson', 'opening.geojson', 'unit.geojson', 'anchor.geojson', 
                               'amenity.geojson', 'footprint.geojson']

    for filename in geojson_files_to_update:
        file_path = os.path.join(directory_path, filename)
        update_ids_in_geojson(file_path, address_uuid, building_uuid, level_uuid)
        print(f"Updated IDs in {filename}")

if __name__ == "__main__":
    main()


Updated IDs in level.geojson
Updated IDs in venue.geojson
Updated IDs in opening.geojson
Updated IDs in unit.geojson
Updated IDs in anchor.geojson
Updated IDs in amenity.geojson
Updated IDs in footprint.geojson


In [81]:
def is_point_inside_polygon(point, polygon):
    """
    Determine if the point is inside the polygon using the ray casting algorithm.
    point: tuple (x, y)
    polygon: list of tuples [(x1, y1), (x2, y2), ...]
    """
    x, y = point
    odd_nodes = False
    j = len(polygon) - 1

    for i in range(len(polygon)):
        xi, yi = polygon[i]
        xj, yj = polygon[j]
        if yi < y and yj >= y or yj < y and yi >= y:
            if xi + (y - yi) / (yj - yi) * (xj - xi) < x:
                odd_nodes = not odd_nodes
        j = i

    return odd_nodes

import json
import os

def is_point_inside_polygon(point, polygon):
    """
    Determine if the point is inside the polygon using the ray casting algorithm.
    point: tuple (x, y)
    polygon: list of tuples [(x1, y1), (x2, y2), ...]
    """
    x, y = point
    odd_nodes = False
    j = len(polygon) - 1

    for i in range(len(polygon)):
        xi, yi = polygon[i]
        xj, yj = polygon[j]
        if yi < y and yj >= y or yj < y and yi >= y:
            if xi + (y - yi) / (yj - yi) * (xj - xi) < x:
                odd_nodes = not odd_nodes
        j = i

    return odd_nodes

def update_unit_id_for_anchors(anchor_file_path, unit_file_path):
    """
    Update the unit_id for each anchor based on which unit it's located in.
    """
    with open(anchor_file_path, 'r') as f:
        anchors = json.load(f)

    with open(unit_file_path, 'r') as f:
        units = json.load(f)

    for anchor in anchors['features']:
        point = tuple(anchor['geometry']['coordinates'])
        for unit in units['features']:
            polygon = unit['geometry']['coordinates'][0][0]
            if is_point_inside_polygon(point, polygon):
                anchor['properties']['unit_id'] = unit['id']
                break

    with open(anchor_file_path, 'w') as f:
        json.dump(anchors, f, indent=4)

def update_unit_ids_for_amenities(amenity_file_path, unit_file_path):
    """
    Update the unit_ids for each amenity based on where its point geometry is located.
    """
    with open(amenity_file_path, 'r') as f:
        amenities = json.load(f)

    with open(unit_file_path, 'r') as f:
        units = json.load(f)

    for amenity in amenities['features']:
        point = tuple(amenity['geometry']['coordinates'])
        for unit in units['features']:
            polygon = unit['geometry']['coordinates'][0][0]
            if is_point_inside_polygon(point, polygon):
                amenity['properties']['unit_ids'] = [unit['id']]
                break

    with open(amenity_file_path, 'w') as f:
        json.dump(amenities, f, indent=4)

def main():
    directory_path = "C:\\Users\\amosk\\GitHub\\Geojson_2_IMDF_Transformer\\Transformed_QGIS_Geojsons"
    
    # Update unit_id for anchors
    update_unit_id_for_anchors(os.path.join(directory_path, 'anchor.geojson'), os.path.join(directory_path, 'unit.geojson'))
    print("Updated unit_id for anchors.")

    # Update unit_ids for amenities
    update_unit_ids_for_amenities(os.path.join(directory_path, 'amenity.geojson'), os.path.join(directory_path, 'unit.geojson'))
    print("Updated unit_ids for amenities.")

if __name__ == "__main__":
    main()


Updated unit_id for anchors.
Updated unit_ids for amenities.


In [83]:
## Just put in some placeholder values

import json

def update_unit_category(unit_file_path):
    """
    Update the category for each unit to "room".
    """
    with open(unit_file_path, 'r') as f:
        units = json.load(f)

    for unit in units['features']:
        unit['properties']['category'] = "room"

    with open(unit_file_path, 'w') as f:
        json.dump(units, f, indent=4)

# Sample usage:
update_unit_category('C:\\Users\\amosk\\GitHub\\Geojson_2_IMDF_Transformer\\QGIS_Geojsons\\unit.geojson')


FileNotFoundError: [Errno 2] No such file or directory: 'path_to_unit.geojson'